Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Sedykh Ivan"
COLLABORATORS = ""

---

In [2]:
import numpy as np

from numpy.testing import assert_allclose


### Часть I. Постройте отражение Хаусхолдера для вектора.

Дан вектор $\mathbf{x}$ и плоскость, заданная вектором нормали $\mathbf{u}$. Преобразование Хаусхолдера отражает $\mathbf{x}$ относительно плоскости.

Матрица преобразований Хаусхолдера:
$$ \mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T $$

Если даны два вектора $\mathbf{x}$ и $\mathbf{y}$ одинаковой длины, поворот, преобразующий $\mathbf{x}$ в $\mathbf{y}$ называется преобразованием Хаусхолдера с
$$ \mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|} $$

Напишите функцию, преобразующую заданный вектор $\mathbf{x} = (x_1, \dots, x_n)$ в $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$, используя преобразование Хаусхолдера.


In [3]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    y = np.zeros_like(vec)
    y[0] = np.linalg.norm(vec)
    d = vec - y
    u = d / np.linalg.norm(d)
    H = np.eye(u.shape[0]) - 2 * u * u.reshape(-1, 1)
    return y, H

Протестируйте свою функцию на следующих примерах:

In [4]:
v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(h @ v1, v, atol=1e-14)
assert_allclose(h @ v, v1, atol=1e-14)

assert_allclose(v1[1:], 0, atol=1e-14)

assert_allclose(h @ h.T, np.eye(3), atol=1e-14)


### Part II. Вычисление $\mathrm{QR}$ - разложения матрицы.

Дана прямоугольная $m\times n$ матрица $\mathbf{A}$. Выполните отражение Хаусхолдера матрицы $\mathbf{H}_1$, преобразующее первый столбец матрицы $\mathbf{A}$ (назовем результат $\mathbf{A}^{(1)}$)

$$ 
\mathbf{H}_1 \mathbf{A} %\begin{pmatrix} \times &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ &amp;&amp; \dots&amp;&amp; \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ \end{pmatrix}
%
\equiv \mathbf{A}^{(1)}\;. 
$$

Теперь рассмотрим нижнюю правую часть матрицы $\mathbf{A}^{(1)}$ и выполним преобразование Хаусхолдера, действующее на 2 столбец $\mathbf{A}$:
$$
\mathbf{H}_2 \mathbf{A}^{(1)} % \begin{pmatrix} \times &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; 0 &amp; \times &amp; \dots &amp; \times \\ &amp;&amp; \dots&amp;&amp; \\ 0 &amp; 0 &amp; \times &amp; \dots &amp; \times \\ \end{pmatrix}
%
\equiv \mathbf{A}^{(2)} \;. 
$$

Повторяя процесс $n$ раз, получим
$$
\mathbf{H}_{n} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;, 
$$

где $\mathbf{R}$ верхняя треугольная матрица. Так как каждая из матриц $\mathbf{H}_k$ ортогональна, таковым будет и их произведение. Обратная от ортогональной также есть матрица ортогональная. Таким образом, алгоритм создает $\mathrm{QR}$ - разложение матрицы $\mathbf{A}$.

Напишите функцию, получающую прямоугольную матрицу $A$ и возвращающую матрицы $Q$ и $R$ --- компоненты $QR$-разложения $A$.


In [5]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    H = np.eye(m)
    for i in range(n):
        x = a1[i:, i]
        _, h = householder(x)
        H1 = np.eye(m)
        H1[i:, i:] = h
        a1 = H1 @ a1
        H = H1 @ H
    Q = H.T
    R = a1
    return Q, R

In [6]:
# можете запустить данную операцию для бооее сжатого вывода: нули вместо `1e-16` и т.д.
np.set_printoptions(suppress=True)

In [7]:
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
aa = a.copy()

q, r = qr_decomp(a)

# check that `qr_decomp` leaves `a` intact
assert_allclose(a, aa, atol=1e-16)

# тестируем, что Q ортогональна
assert_allclose(q @ q.T, np.eye(5), atol=1e-14)

# проверяем разложение
assert_allclose(q @ r, a, atol=1e-14)

Теперь сравните ваше разложение с разложением, полученным библиотечной функцией (содержащей соответствующие функции библиотеки LAPACK)

In [8]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a,
                atol=1e-14)

Проверьте, согласуются ли ваши матрицы `q` и `r` с `qq` и `rr`. Объясните результат.
Оставьте пояснения в этой ячейке.

In [13]:
np.isclose(abs(q), abs(qq))

array([[ True,  True,  True, False, False],
       [ True,  True,  True, False, False],
       [ True,  True,  True, False, False],
       [ True,  True,  True, False, False],
       [ True,  True,  True, False, False]])

In [14]:
np.allclose(abs(r), abs(rr))

True

m > n (5 > 3) so it is economic QR, so 2 last columns of Q don't matter.


### Часть III. Безматричная реализация.

Отметим необычную структуру матрицы Хаусхолдера: матрица поворота $\mathbf{H}$ полностью характеризуется вектором отражения $\mathbf{u}$. Заметим, также, что вычислительная сложность операции отражения матрицы сильно зависит от порядка операций:
$$ \left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A} \qquad \textrm{is } O(m^2 n)\;, $$

тогда как $$ \mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn) $$

Таким образом, следует избегать формирований матриц $\mathbf{H}$. Вместо этого можно сохранять сами векторы отражения $\mathbf{u}$ и производить умножение произвольной матрицы на $\mathbf{Q}^T$, например, как отдельную функцию (класс).

Напишите функцию, выполняющую QR - разложение матрицы без формирования матриц $\mathbf{H}$ и возвращающую матрицу $\mathbf{R}$, а также вектора отражений Хаусхолдера.


Напишите вторую функцию, которая использует вектора отражений, полученных предыдущей функцией, для вычисления произведения $Q^T B$ для заданной матрицы B. Убедитесь, что вы добавили достаточно комментариев, следующих вашим выкладкам. 



In [15]:
def get_u_householder(vec):
    """Construct a Householder reflection vector to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    u : array of floats, shape (n,)
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    y = np.zeros_like(vec)
    y[0] = np.linalg.norm(vec)
    d = vec - y
    u = d / np.linalg.norm(d)
    return u


def qr_nomatrix(a):
    """Form QR decomposition of `a` via successive Householder reflections.
    
    Parameters
    ----------
    a : ndarray
        Input matrix
    
    Returns
    -------
    R : ndarray
        Upper triangular matrix of the QR decomposition
    
    U : ndarray
        Columns store successive Householder reflectors: `U[j:, j]` stores
        the Householder reflector for reducing the `j`-th column.
        
    See Also
    --------
    householder_apply : apply Householder reflectors stored in `U` to `a`.
    
    """
    
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    U = np.zeros_like(a1)
    for i in range(n):
        x = a1[i:, i]
        u = get_u_householder(x)
        U[i:, i] = u
        fix_a1 = np.zeros_like(a1)
        fix_a1[i:, i:] = -2 * u.reshape(-1, 1) * (u.reshape(1, -1) @ a1[i:, i:])
        a1 += fix_a1
    R = a1
    return R, U
    

    
def householder_apply(b, uu):
    """Apply the Householder reflectors stored in `uu` to `b`.
    
    The result is equivalent to
    >>> r, q = qr_decomp(a)
    >>> q.T @ b
    
    Parameters
    ----------
    b : ndarray
        Input matrix
    uu : ndarray
        Householder reflectors: `U[j:, j]` is the reflection vector
        for transforming the `j`-th column of `a`.
        
    Returns
    -------
    r : ndarray
        The result of applying the reflectors to `b`. Equivalent to
        computing `q.T @ b`.

    See Also
    --------
    qr_decomp
    
    """

    a1 = np.array(b, copy=True, dtype=float)
    m, n = a1.shape
    
    for i in range(n):
        u = uu[i:, i]
        fix_a1 = np.zeros_like(a1)
        fix_a1[i:, i:] = -2 * u.reshape(-1, 1) * (u.reshape(1, -1) @ a1[i:, i:])
        a1 += fix_a1
    R = a1
    return R

In [16]:
rndm = np.random.RandomState(1223)

a = rndm.uniform(size=(5, 5))
R1, U = qr_nomatrix(a)
R2 = householder_apply(a, U)
R_lib = qr(a)[1]
assert_allclose(R1, R2, atol=1e-14)


In [17]:
# Check consistency with the scipy library decomposition. Allow for sign differences

conds = [np.allclose(R2[i, :], R_lib[i, :], atol=1e-14) or
         np.allclose(R2[i, :], -R_lib[i, :], atol=1e-14) for i in range(5)]
assert False not in conds


In [18]:
# More testing here, keep this cell intact
